In [1]:
# Importēt nepieciešamās bibliotēkas

import gzip
# from rdflib.namespace import RDF
# from rdflib import Dataset, Namespace, RDF, RDFS, URIRef, Literal, BNode
from collections import Counter

In [2]:
# Funkcija, kas izveido RDF četrinieku sarakstu no .gz faila vai failiem
# Jāpadod faila vārds un Schema.org klases vārds, piemēram, School

def parse_gz_files(filenames, schema_org_class_name):
    current_subject_has_class = False
    quads = []
    
    for filename in filenames:
        with gzip.open(filename, 'rt', encoding='utf-8') as f:
            for line in f:

                line = line.strip()
                if not line or line.startswith('#'):
                    continue
                
                try:
                    s, p, o, g, _ = line.split()
                except ValueError:
                    continue

                # Datu kopā RDF četriniekiem ir šāda struktūra:
                # Sākumā nāk četrinieks, kas norāda aprakstamās entītijas tipu
                # Tam seko pārējie četrinieki, kas apraksta entītijas īpašības
                # Tātad, datu apstrāde notiek blokos: sākumā nolasām tipu, tad nolasām īpašības

                # Datu kopa satur ne tikai entītijas ar tipu X, bet arī citas
                # Mēs gribam analizēt tikai entītijas ar tipu X
                # Lai noskaidrotu tipu, mēs nolasām predikātu, kas apzīmē tipu, un pārbaudām, vai tā vērtība atbilst schema.org tipam X
                # Ja tips nav X, tad datu bloks tiek ignorēts

                if p == "<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>":
                    if o == f"<http://schema.org/{schema_org_class_name}>":
                        current_subject_has_class = True
                    else:
                        current_subject_has_class = False

                if current_subject_has_class:
                    quads.append((s, p, o, g))
    
    return quads


In [3]:
# Iegūt četriniekus klasei ShoppingCenter 2024. gada datu kopā

shopping_center_quads = {}
shopping_center_quads["2024"] = parse_gz_files(["../shopping_center_2024_p1.gz", "shopping_center_2024_p2.gz"], "ShoppingCenter")

KeyboardInterrupt: 

In [5]:


for year, quads in shopping_center_quads.items():
    # Top 10 predikāti (predicates) entītijām ar tipu School
    
    predicate_counter = Counter()

    for s, p, o, graph in quads:
        predicate_counter[p] += 1

    top_predicates = predicate_counter.most_common(10)

    print(f"\nTop 10 predikāti {year}:")

    # Īpašība; entītiju skaits, kas izmanto; procents no kopējā entītiju skaita
    # Tā kā katra entītija sākas ar tipa predikātu, tipa predikātu skaitu var uzskatīt par entītiju skaitu
    for p, count in top_predicates:
        print(p, count, str(round(100 * (count / (predicate_counter["<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>"])), 2)) + "%")


Top 10 predikāti 2024:
<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> 180907 100.0%
<http://schema.org/image> 153896 85.07%
<http://schema.org/address> 152417 84.25%
<http://schema.org/url> 117387 64.89%
<http://schema.org/openingHoursSpecification> 90660 50.11%
<http://schema.org/name> 87362 48.29%
<http://schema.org/sameAs> 84264 46.58%
<http://schema.org/geo> 66517 36.77%
<http://schema.org/telephone> 55623 30.75%
<http://schema.org/containsPlace> 28816 15.93%
